In [1]:
from glob import glob
import cv2
import pandas as pd
import json
import numpy as np
import os
from tqdm import tqdm

In [2]:
train_video = sorted(glob("/data/competition/Another/Busan/Competition_1/행위/행위/*/*.mp4"))
train_json = sorted(glob("/data/competition/Another/Busan/Competition_1/행위/행위/*/*.json"))

In [3]:
j = open(train_video[0].replace("mp4", "json"))
j = json.load(j)

In [4]:
def Vid2Bbox(vid_list):
    
    for i in tqdm(vid_list):
        
        j = open(i.replace("mp4", "json"))
        j = json.load(j)
        
        # Video Info
        fps = j["info"]["fps"]
        frame = j["info"]["frame"]
        height = j["info"]["height"]
        width = j["info"]["width"]
        cap = cv2.VideoCapture(i)
        
        event_num = 0
        
        for num, k in enumerate(j["events"]):
            # Make Folder
            if not os.path.exists(i.replace(".mp4", f"_{num}")):
                os.makedirs(i.replace(".mp4", f"_{num}"))
            event_num = event_num + 1
        n_f = int(j["annotations"][0]["cur_frame"])
        m = 0
        for l in range(len(j["annotations"])):
            minx, miny = j["annotations"][l]["bbox"][0]
            minx = int(minx)
            miny = int(miny)
            maxx, maxy = j["annotations"][l]["bbox"][1]
            maxx = int(maxx)
            maxy = int(maxy)
            num_frame = j["annotations"][l]["cur_frame"]
            
            if n_f != int(num_frame):
                con_frame = int(n_f)
                while(True):
                    con_frame = con_frame + 1
                    if len(glob(i.replace(".mp4", f"_{m}/*"))) > 65:
                        break
                    if not os.path.exists(i.replace(".mp4", f"_{m}")):
                        break
                    frame = np.zeros((256, 256,3))
                    cv2.imwrite(i.replace(".mp4", f"_{m}") + '/' + str(con_frame).zfill(5) + '.png', frame)
                m = m + 1
                n_f = int(num_frame) + 1
                cap = cv2.VideoCapture(i)
            else:
                n_f = n_f + 1

            # Extract BoundingBox
            cap.set(cv2.CAP_PROP_FRAME_COUNT, num_frame)
            ret,frame = cap.read()
            if ret is False:
                break

            frame = cv2.resize(frame[miny:maxy, minx:maxx], (256, 256))
            cv2.imwrite(i.replace(".mp4", f"_{m}") + '/' + str(num_frame).zfill(5) + '.png', frame)
                
        con_frame = int(n_f)
        while(True):
            con_frame = con_frame + 1
            if len(glob(i.replace(".mp4", f"_{m}/*"))) > 65:
                break
            if not os.path.exists(i.replace(".mp4", f"_{m}")):
                break
            frame = np.zeros((256, 256,3))
            cv2.imwrite(i.replace(".mp4", f"_{m}") + '/' + str(con_frame).zfill(5) + '.png', frame)
        
        cap.release()

In [5]:
Vid2Bbox(train_video)

100%|██████████| 1458/1458 [36:26<00:00,  1.50s/it]
